In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import pandas as pd
from numpy import dtype
!pip install pymysql
import sqlalchemy as db
import tarfile
import yaml
import tensorflow_datasets as tfds
import keras
from keras.layers import Input, Dense, concatenate
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [0]:
type_mappings = {dtype('int64'): 'numerical', dtype('float64'): 'numerical', dtype('object'): 'string', dtype('bool'): 'binary'}

In [0]:
url = 'mysql+pymysql://dataset:Aquinoentras22@@51.15.95.84/socialhuman'  #or get from a kafka topic....
table = 'emails'  #this could be a view to pick up rolling 6 months of data to support re-training
engine = db.create_engine(url)
query="select senderaddress, subject,JSON_SEARCH(folderlabel,'one', 'CATEGORY_SOCIAL') IS NOT null AS issocial from socialhuman.emails WHERE (JSON_SEARCH(folderlabel,'one', 'CATEGORY_SOCIAL') IS NOT NULL OR JSON_SEARCH(folderlabel,'one', 'CATEGORY_UPDATES') IS NOT NULL)  ORDER BY RAND() LIMIT 1000"
df = pd.read_sql_query(query, engine)
df_test = pd.read_sql_query("select senderaddress,subject from socialhuman.emails WHERE (JSON_SEARCH(folderlabel,'one', 'CATEGORY_SOCIAL') IS NOT NULL OR JSON_SEARCH(folderlabel,'one', 'CATEGORY_UPDATES') IS NOT NULL)  ORDER BY RAND() LIMIT 10", engine)
df['issocial'] = df['issocial'].astype('bool')
target = 'issocial'
df.head()

,senderaddress,subject,issocial
0,Twitter <info@twitter.com>,"soundcloudify, no te prives de compartir",True
1,"""Jordan, GoDaddy Guide Team"" <donotreply@godad...",Urgent: You're about to lose carmagazine.xyz o...,False
2,LinkedIn <notifications-noreply@linkedin.com>,Congratulate Laura Casal Fernández for...,True
3,"""Jack @ VYPER ⚡️"" <jack@vyper.io>",🔍 How to easily track your customers journey w...,False
4,Facebook <notification@facebookmail.com>,Consulta a quién le ha gustado tu página,True


In [0]:
#code copied from ludwig as concept proof for a pseudo auto definition if dataset change, not apply now
input_features = []


for col in df.columns:
 if col == target:
   continue
 n = df[col].name
 t = df[col].dtype
 input_features.append({'name': n, 'type': type_mappings[t]})

print(input_features)
model_def = {}
model_def['input_features'] = input_features
model_def['output_features'] =  [{'name': 'issocial', 'type': 'binary'}]
model_def['training']=  {'batch_size': 100, 'epochs':1    }  

[{'name': 'senderaddress', 'type': 'string'}, {'name': 'subject', 'type': 'string'}]


In [0]:
lines_dataset = tf.data.TextLineDataset('/content/drive/My Drive/Colab Notebooks/sample_text.txt')
for line in lines_dataset.take(5):
  print(line.numpy())

b'dominio*****||*****seccion*****||*****url*****||*****title*****||*****tags*****||*****pretitle*****||*****posttitle*****||*****imagen_url*****||*****imagen_descripcion*****||*****texto'
b'www.elconfidencial.com*****||*****mundo*****||*****https://www.elconfidencial.com/mundo/europa/2019-09-03/ultima-estocada-a-salvini-el-79-de-los-inscritos-del-m5s-apoya-la-nueva-coalicion_2208291/*****||*****Las bases del M5S apoyan por mayor\xc3\xada la coalici\xc3\xb3n de gobierno en Italia con el PD*****||*****Socialdemocracia|Movimiento 5 Estrellas|*****||*****\xc3\x9aLTIMA ESTOCADA A SALVINI*****||*****El 79% de los inscritos del Movimiento Cinco Estrellas (M5S) vot\xc3\xb3 hoy a favor del Gobierno en coalici\xc3\xb3n con el Partido Dem\xc3\xb3crata (PD), con Giuseppe Conte como primer ministro*****||*****https://www.ecestaticos.com/imagestatic/clipping/8b3/17c/8b317c51beca47d40dc5be6c8388e01a/el-gobierno-de-conte-ii-dara-un-respiro-a-italia-pero-no-lograra-frenar-a-salvini.jpg?mtime=1567021325

In [0]:
training_samples = 28
INPUTS = ['seccion', 'title']
INLEN = [255,500]
OUTPUTS = ['tags']
OUTLEN = [255]
def parsing(line):
  """
  Parse the file and split by category
  """
  splitted = tf.strings.split(line, sep='*****||*****', maxsplit=9) 
  #tags = tf.strings.split(splitted[4], '|')
  return (splitted[1], splitted[3]), splitted[4]




In [0]:
parsed_data = lines_dataset.map(parsing)

In [0]:
for line in parsed_data.take(5):
  print(line)

((<tf.Tensor: shape=(), dtype=string, numpy=b'seccion'>, <tf.Tensor: shape=(), dtype=string, numpy=b'title'>), <tf.Tensor: shape=(), dtype=string, numpy=b'tags'>)
((<tf.Tensor: shape=(), dtype=string, numpy=b'mundo'>, <tf.Tensor: shape=(), dtype=string, numpy=b'Las bases del M5S apoyan por mayor\xc3\xada la coalici\xc3\xb3n de gobierno en Italia con el PD'>), <tf.Tensor: shape=(), dtype=string, numpy=b'Socialdemocracia|Movimiento 5 Estrellas|'>)
((<tf.Tensor: shape=(), dtype=string, numpy=b'alma-corazon-vida'>, <tf.Tensor: shape=(), dtype=string, numpy=b'C\xc3\xb3mo identificar r\xc3\xa1pido una mentira en un mensaje de texto'>), <tf.Tensor: shape=(), dtype=string, numpy=b'Virales|Sin censura|Sexo|'>)
((<tf.Tensor: shape=(), dtype=string, numpy=b'mundo'>, <tf.Tensor: shape=(), dtype=string, numpy=b'"Vamos a por ti": la ONU acusa a Arabia Saud\xc3\xad del crimen del periodista Khashoggi'>), <tf.Tensor: shape=(), dtype=string, numpy=b'ONU|Derechos humanos|Arabia Saud\xc3\xad|Monarqu\xc3\

In [0]:
##pte ver que hace num_words
tags_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=255, split='|')
section_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=255)
title_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=500)
tags_vocabulary_set = set()
section_vocabulary_set = set()
title_vocabulary_set = set()

In [0]:

for (section, title), tags in parsed_data.skip(1).take(training_samples):
  tags_vocabulary_set.update([tags.numpy().decode()])
  section_vocabulary_set.update([section.numpy().decode()])
  title_vocabulary_set.update([title.numpy().decode()])

In [0]:

tags_tokenizer.fit_on_texts(list(tags_vocabulary_set))
section_tokenizer.fit_on_texts(list(section_vocabulary_set))
title_tokenizer.fit_on_texts(list(title_vocabulary_set))

In [0]:
print(section_tokenizer)

In [0]:
# Preparing the data pipeline
def encoder_data(text_tensor, label):
  """
  Uses the trained encoder to convert the text sequences into arrays
  Note: The tag array contain the labeled data
  """
  section = section_tokenizer.texts_to_matrix([text_tensor[0].numpy().decode()])[0] 
  title = title_tokenizer.texts_to_matrix([text_tensor[1].numpy().decode()])[0]  
  tag = tags_tokenizer.texts_to_matrix([label.numpy().decode()])[0]
  return section, title, tag

def encode_map_fn(text_tensor, label):
  result = tf.py_function(encoder_data, inp=[text_tensor, label], Tout=(tf.int64, tf.int64, tf.int64))
  return {'seccion': result[0], 'title':result[1]}, {'tags':result[2]}


In [0]:
# Skipping the first line and using a batchsize of 14
train_data = parsed_data.map(encode_map_fn).skip(1)


In [0]:
print(training_samples)

28


In [0]:
train_data = train_data.take(training_samples).padded_batch(14, padded_shapes=({'seccion':[None], 'title':[None]},{'tags':[None]}))

In [0]:
for i in train_data:
  print(i[0]['seccion'].shape, i[0]['title'].shape, i[1]['tags'].shape)

(14, 255) (14, 500) (14, 255)
(14, 255) (14, 500) (14, 255)


In [0]:
# Creating the model

section_input = tf.keras.layers.Input(shape=(255,), name='seccion', dtype=tf.int64)
title_input = tf.keras.layers.Input(shape=(500,), name='title', dtype=tf.int64)
section_emb = tf.keras.layers.Embedding(1024,32)(section_input)
title_emb = tf.keras.layers.Embedding(1024,32)(title_input)
mixed = tf.keras.layers.concatenate([tf.keras.layers.Flatten()(section_emb), tf.keras.layers.Flatten()(title_emb)])
out = tf.keras.layers.Dense(255, activation='sigmoid', name='tags')(mixed)
model = tf.keras.models.Model([section_input,title_input], [out])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seccion (InputLayer)            [(None, 255)]        0                                            
__________________________________________________________________________________________________
title (InputLayer)              [(None, 500)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 255, 32)      32768       seccion[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 500, 32)      32768       title[0][0]                      
______________________________________________________________________________________________

In [0]:
model.compile(loss='binary_crossentropy')
model.fit(train_data, epochs=50)

Epoch 1/50
2/2 [==============================] - 0s 60ms/step - loss: 0.4328
Epoch 2/50
2/2 [==============================] - 0s 50ms/step - loss: 0.1012
Epoch 3/50
2/2 [==============================] - 0s 48ms/step - loss: 0.0822
Epoch 4/50
2/2 [==============================] - 0s 52ms/step - loss: 0.0769
Epoch 5/50
2/2 [==============================] - 0s 50ms/step - loss: 0.0744
Epoch 6/50
2/2 [==============================] - 0s 48ms/step - loss: 0.0730
Epoch 7/50
2/2 [==============================] - 0s 51ms/step - loss: 0.0721
Epoch 8/50
2/2 [==============================] - 0s 50ms/step - loss: 0.0714
Epoch 9/50
2/2 [==============================] - 0s 48ms/step - loss: 0.0708
Epoch 10/50
2/2 [==============================] - 0s 50ms/step - loss: 0.0703
Epoch 11/50
2/2 [==============================] - 0s 49ms/step - loss: 0.0698
Epoch 12/50
2/2 [==============================] - 0s 51ms/step - loss: 0.0692
Epoch 13/50
2/2 [==============================] - 0s 48ms/st

In [0]:
class MyWordEmbeddingLayerLayer(tf.keras.layers.Layer):
  def __init__(self, tokenizer, output_size=255):
    super(MyWordEmbeddingLayerLayer, self).__init__()
    key_values = tf.lookup.KeyValueTensorInitializer(list(tokenizer.word_index.keys()),
                                                     list(tokenizer.word_index.values()),
                                                     key_dtype=tf.string, value_dtype=tf.int64)
    self.table = tf.lookup.StaticVocabularyTable(key_values,10)
    self.lookup = tf.function(self.table.lookup)
    self.output_size = output_size

  def call(self, input_string):
    queries = tf.ragged.map_flat_values(tf.strings.split, input_string)
    word_embeddings = tf.ragged.map_flat_values(lambda x: self.lookup(x), queries).to_tensor(default_value=0)
    word_embeddings = tf.pad(word_embeddings, [[0,0],[0,0], [0,self.output_size]], 'CONSTANT')[:,:,:self.output_size]
    return tf.reshape(word_embeddings, [queries.nrows(), self.output_size])

In [0]:
section_input = tf.keras.layers.Input(shape=(1,), name='seccion', dtype=tf.string)
title_input = tf.keras.layers.Input(shape=(1,), name='title', dtype=tf.string)
section_encode = MyWordEmbeddingLayerLayer(section_tokenizer)(section_input)
title_encode = MyWordEmbeddingLayerLayer(title_tokenizer, 500)(title_input)
#test_model = tf.keras.models.Model([section_input, title_input], [section_encode, title_encode])
#test_model.summary()
#test_model.predict([['mundo'],['la socialdemocracia']])[0].shape, test_model.predict([['mundo'],['la socialdemocracia']])[1].shape

In [0]:
new_outputs = model({'seccion':section_encode, 'title':title_encode})
new_model = tf.keras.models.Model(inputs={'seccion':section_input, 'title':title_input}, outputs={'tags':new_outputs})
new_model.summary()
#new_model.predict({'seccion':'mundo', 'title':'la socialdemocracia'})

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seccion (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
title (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
my_word_embedding_layer_layer ( (None, 255)          0           seccion[0][0]                    
__________________________________________________________________________________________________
my_word_embedding_layer_layer_1 (None, 500)          0           title[0][0]                      
____________________________________________________________________________________________

In [0]:
import pickle 
tf.saved_model.save(new_model, "/content/drive/My Drive/modelos/servings/textclassifier/1")

# saving the tokenizers
with open('section_tokenizer.pickle', 'wb') as handle:
    pickle.dump(section_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('title_tokenizer.pickle', 'wb') as handle:
    pickle.dump(title_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('tags_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tags_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/modelos/servings/textclassifier/1/assets
